In [45]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pandas.tseries.offsets import DateOffset
from sklearn import metrics
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# the pandas webreader provides remote data access to apis
import pandas_datareader as webreader

date_today = "2022-11-10"
date_start = "2010-01-01"

# set the symbol to bitcoin-usd quotes
# for more symbols check yahoo.finance.com
symbol = "BTC-USD"

In [3]:
# now we will send the request to the yahoo finance api endpoint
df = webreader.DataReader(symbol, start=date_start, end=date_today, data_source="yahoo")
df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,21056800,457.334015
2014-09-18,456.859985,413.104004,456.859985,424.440002,34483200,424.440002
2014-09-19,427.834991,384.532013,424.102997,394.795990,37919700,394.795990
2014-09-20,423.295990,389.882996,394.673004,408.903992,36863600,408.903992
2014-09-21,412.425995,393.181000,408.084991,398.821014,26580100,398.821014


In [4]:
df['Pct Change'] = df['Close'].pct_change()

In [5]:
df['50 SMA'] = ta.SMA(df['Close'], timeperiod=50)
df['200 SMA'] = ta.SMA(df['Close'], timeperiod=200)
df['10 EMA'] = ta.EMA(df['Close'], timeperiod=10)
df['20 EMA'] = ta.EMA(df['Close'], timeperiod=20)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

In [47]:
df['Signal'] = 0
df.loc[(df['Pct Change'] >=0), 'Signal'] = 1
df.loc[(df['Pct Change'] <0), 'Signal'] = 0

In [48]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,Pct Change,50 SMA,200 SMA,10 EMA,20 EMA,RSI,Signal
Date,,,,,,,,,,,,,
2022-11-07,21053.246094,20489.972656,20924.621094,20602.816406,53510852236,20602.816406,-0.015467,19683.553633,23881.031875,20634.032692,20341.873001,55.038707,0
2022-11-08,20664.607422,17603.544922,20600.671875,18541.271484,118992465607,18541.271484,-0.100061,19663.496484,23775.036631,20253.530654,20170.387142,35.913352,0
2022-11-09,18590.458984,15682.692383,18543.761719,15880.780273,102905151606,15880.780273,-0.143490,19603.296309,23657.006880,19458.485130,19761.853155,24.217638,0
2022-11-10,18054.312500,15834.018555,15883.158203,17586.771484,83202283721,17586.771484,0.107425,19584.083730,23547.594272,19118.173558,19554.702520,38.131300,1
2022-11-11,17650.943359,16543.482422,17583.251953,17034.292969,55871616488,17034.292969,-0.031414,19536.498574,23430.474194,18739.286178,19314.663515,35.836612,0


In [49]:
X = df[['Close', 'Volume', 'Pct Change', '50 SMA', '200 SMA', 'RSI']].shift().dropna()
y = df['Signal'].dropna()


In [50]:
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=12)

In [51]:
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [52]:
X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]

In [53]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [54]:
svm_model = SVC()
svm_model = svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [ ]:
svm_testing_report = classification_report(y_test, svm_pred)
print(svm_testing_report)

              precision    recall  f1-score   support

           0       0.29      0.00      0.01      1110
           1       0.54      0.99      0.70      1301

    accuracy                           0.54      2411
   macro avg       0.41      0.50      0.35      2411
weighted avg       0.42      0.54      0.38      2411

